In [64]:
# ! pip install pydantic
# !pip install pydantic[email]

In [76]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, Optional, Literal
import os
from pydantic import BaseModel, EmailStr, Field

load_dotenv('EnvVars.txt')

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_API_KEY'] = os.getenv('HUGGINGFACEHUB_ACCESS_TOKEN')


In [66]:
model = ChatOpenAI(model = 'gpt-4')

In [115]:
class Student(BaseModel): # for data validation
    name : str = "sathwik"
    age : Optional[int] = None
    email: Optional[EmailStr] = None
    cgpa : Optional[float] = Field(gt = 0, lt =10, default = 5, description = 'Decimal value representing the performance in academics') 

In [116]:
new_student = {'age':'32'} # its doing implicit type coersion, pydantic is samrt enough to identiffy that

In [117]:
student = Student(**new_student)

In [118]:
print(student)

name='sathwik' age=32 email=None cgpa=5


In [119]:
new_student = {'email': 'sathwik238@gmail.com', 'cgpa':9}

In [120]:
student = Student(**new_student)

In [125]:
student_dict = dict(student)

In [132]:
student_dict

{'name': 'sathwik', 'age': None, 'email': 'sathwik238@gmail.com', 'cgpa': 9.0}

In [133]:
student_dict['cgpa']

9.0

In [140]:
# schema
class Review(BaseModel):
    
    key_themes : list[str] = Field(description = "Write down key themes discussed int he review")
    summary : str = Field(description = "A brief summary odf the review") 
    sentiment : Literal["pos","neg"] = Field(description = "return sentiment of the review") 
    pros : list[str] = Field(description = "write down all the pros inside a list, like good battery, best screen, 1 word for the level of proness and one word for the attribute which is being boasted")
    cons : Optional[list[str]] = Field(description = "it is optional, write down all the cons inside a list,like bad battery, worst screen, 1 word for the level of conness and one word for the attribute which is being bad ")
    
     

In [141]:
structured_model = model.with_structured_output(Review)

/opt/conda/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [142]:
result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.""")

In [143]:
dict(result)

{'key_themes': ['Processor',
  'Battery',
  'Charging',
  'S-Pen',
  'Camera',
  'Size and Weight',
  'Bloatware',
  'Price'],
 'summary': "The reviewer praises the Samsung Galaxy S24 Ultra for its fast processor, long-lasting battery, S-Pen integration and high-quality camera, but finds the phone's size and amount of bloatware disappointing. The high price is also considered a drawback.",
 'sentiment': 'pos',
 'pros': ['Fast Processor',
  'Long-lasting Battery',
  'Quick Charging',
  'S-Pen Integration',
  'High-Quality Camera'],
 'cons': ['Heavy', 'Bulky', 'Bloatware', 'Expensive']}